In [58]:
# from google.colab import files
# uploaded = files.upload()

In [59]:
from pandas import DataFrame
import pandas

# convert series to supervised learning
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = pandas.concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

In [60]:
from pandas import read_csv
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

dataset = read_csv('data/pm2.5_non_normalised.csv', header=0, index_col=0)

In [61]:
import copy
import os
from math import sqrt
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from sklearn.metrics import mean_squared_error
import numpy as np

def grid_search(dataset, lookback, filters, kernels, split_percentage):

    scores = list()
  
    for lookback_period in lookback:
        values = series_to_supervised(dataset, lookback_period, 1).values

        n_train_hours = (int)(split_percentage * len(values))
        train = values[:n_train_hours, :]
        test = values[n_train_hours:, :]

        train_X, train_y = train[:, :-1], train[:, -1]
        test_X, test_y = test[:, :-1], test[:, -1]
        # reshape input to be 3D [samples, timesteps, features]
        train_X = train_X.reshape((train_X.shape[0], lookback_period, 1))
        test_X = test_X.reshape((test_X.shape[0], lookback_period, 1))

        for number_of_filters in filters:      
            for kernel_size in kernels:
                if (kernel_size > number_of_filters):
                    continue
            
                model = Sequential()
                model.add(Conv1D(filters=number_of_filters, kernel_size=kernel_size, activation='relu', input_shape=(lookback_period, 1)))
                #model.add(MaxPooling1D(pool_size=2))
                model.add(Flatten())

                model.add(Dense(1))
                model.compile(loss='mae', optimizer='adam')

                name = str(lookback_period)+'_'+str(number_of_filters)+'_'+str(kernel_size)
                print("Starting training for : ", name)

                history = model.fit(train_X, train_y, epochs=100, batch_size=72, validation_data=(test_X, test_y), verbose=2, shuffle=False)
                yhat = model.predict(test_X)

                np.save('CNN_runtime/history/' + name, history)
                np.save('CNN_runtime/test_y/' + name, test_y)
                np.save('CNN_runtime/yhat/' + name, yhat)
                model.save('CNN_runtime/model/' + name)

                rmse = mean_squared_error(yhat, test_y)
                print("RMSE for ", name, " = " , rmse)

                scores.append([rmse, lookback_period, number_of_filters, kernel_size])
                np.save('CNN_scores', np.array(scores))


    return scores

In [62]:
filters = [2,4,16]
lookback = [3,5,7]
kernels = [1,2,3]

split_percentage = 0.8

scores = grid_search(dataset, lookback, filters, kernels, split_percentage)
print(scores)
np.save('CNN_scores', np.array(scores))

Starting training for :  3_2_1
Train on 35036 samples, validate on 8760 samples
Epoch 1/100
 - 1s - loss: 28.2666 - val_loss: 18.7818
Epoch 2/100
 - 1s - loss: 19.3064 - val_loss: 18.2695
Epoch 3/100
 - 1s - loss: 18.7314 - val_loss: 17.6352
Epoch 4/100
 - 1s - loss: 18.0597 - val_loss: 16.9107
Epoch 5/100
 - 1s - loss: 17.3252 - val_loss: 16.1235
Epoch 6/100
 - 1s - loss: 16.5469 - val_loss: 15.3066
Epoch 7/100
 - 1s - loss: 15.7606 - val_loss: 14.4910
Epoch 8/100
 - 1s - loss: 15.0523 - val_loss: 13.8312
Epoch 9/100
 - 1s - loss: 14.5321 - val_loss: 13.3599
Epoch 10/100
 - 1s - loss: 14.1875 - val_loss: 13.0357
Epoch 11/100
 - 1s - loss: 13.9524 - val_loss: 12.8235
Epoch 12/100
 - 1s - loss: 13.7923 - val_loss: 12.6865
Epoch 13/100
 - 1s - loss: 13.6790 - val_loss: 12.5679
Epoch 14/100
 - 1s - loss: 13.5937 - val_loss: 12.4887
Epoch 15/100
 - 1s - loss: 13.5268 - val_loss: 12.4311
Epoch 16/100
 - 1s - loss: 13.4758 - val_loss: 12.3854
Epoch 17/100
 - 1s - loss: 13.4351 - val_loss: 12

Epoch 47/100
 - 1s - loss: 13.2447 - val_loss: 12.1408
Epoch 48/100
 - 1s - loss: 13.2447 - val_loss: 12.1406
Epoch 49/100
 - 1s - loss: 13.2446 - val_loss: 12.1414
Epoch 50/100
 - 1s - loss: 13.2443 - val_loss: 12.1412
Epoch 51/100
 - 1s - loss: 13.2449 - val_loss: 12.1412
Epoch 52/100
 - 1s - loss: 13.2449 - val_loss: 12.1411
Epoch 53/100
 - 1s - loss: 13.2447 - val_loss: 12.1419
Epoch 54/100
 - 1s - loss: 13.2451 - val_loss: 12.1428
Epoch 55/100
 - 1s - loss: 13.2447 - val_loss: 12.1411
Epoch 56/100
 - 1s - loss: 13.2446 - val_loss: 12.1414
Epoch 57/100
 - 1s - loss: 13.2448 - val_loss: 12.1420
Epoch 58/100
 - 1s - loss: 13.2444 - val_loss: 12.1414
Epoch 59/100
 - 1s - loss: 13.2448 - val_loss: 12.1421
Epoch 60/100
 - 1s - loss: 13.2445 - val_loss: 12.1415
Epoch 61/100
 - 1s - loss: 13.2447 - val_loss: 12.1415
Epoch 62/100
 - 1s - loss: 13.2443 - val_loss: 12.1413
Epoch 63/100
 - 1s - loss: 13.2446 - val_loss: 12.1414
Epoch 64/100
 - 1s - loss: 13.2446 - val_loss: 12.1414
Epoch 65/1

Epoch 94/100
 - 1s - loss: 13.3289 - val_loss: 12.2278
Epoch 95/100
 - 1s - loss: 13.3285 - val_loss: 12.2280
Epoch 96/100
 - 1s - loss: 13.3288 - val_loss: 12.2280
Epoch 97/100
 - 1s - loss: 13.3286 - val_loss: 12.2279
Epoch 98/100
 - 1s - loss: 13.3290 - val_loss: 12.2276
Epoch 99/100
 - 1s - loss: 13.3290 - val_loss: 12.2277
Epoch 100/100
 - 1s - loss: 13.3295 - val_loss: 12.2278
RMSE for  3_4_1  =  604.776651997436
Starting training for :  3_4_2
Train on 35036 samples, validate on 8760 samples
Epoch 1/100
 - 1s - loss: 18.5505 - val_loss: 14.8982
Epoch 2/100
 - 1s - loss: 15.1797 - val_loss: 13.7814
Epoch 3/100
 - 1s - loss: 14.1682 - val_loss: 12.8558
Epoch 4/100
 - 1s - loss: 13.5482 - val_loss: 12.4113
Epoch 5/100
 - 1s - loss: 13.3718 - val_loss: 12.2975
Epoch 6/100
 - 1s - loss: 13.3397 - val_loss: 12.2568
Epoch 7/100
 - 1s - loss: 13.3352 - val_loss: 12.2460
Epoch 8/100
 - 1s - loss: 13.3360 - val_loss: 12.2380
Epoch 9/100
 - 1s - loss: 13.3353 - val_loss: 12.2355
Epoch 10/10

Epoch 39/100
 - 1s - loss: 13.1900 - val_loss: 12.1047
Epoch 40/100
 - 1s - loss: 13.1872 - val_loss: 12.1071
Epoch 41/100
 - 1s - loss: 13.1880 - val_loss: 12.0927
Epoch 42/100
 - 1s - loss: 13.1884 - val_loss: 12.1138
Epoch 43/100
 - 1s - loss: 13.1868 - val_loss: 12.0911
Epoch 44/100
 - 1s - loss: 13.1891 - val_loss: 12.0916
Epoch 45/100
 - 1s - loss: 13.1832 - val_loss: 12.1305
Epoch 46/100
 - 1s - loss: 13.1849 - val_loss: 12.1182
Epoch 47/100
 - 1s - loss: 13.1811 - val_loss: 12.0869
Epoch 48/100
 - 1s - loss: 13.1849 - val_loss: 12.0954
Epoch 49/100
 - 1s - loss: 13.1874 - val_loss: 12.0920
Epoch 50/100
 - 1s - loss: 13.1819 - val_loss: 12.0959
Epoch 51/100
 - 1s - loss: 13.1822 - val_loss: 12.1078
Epoch 52/100
 - 1s - loss: 13.1733 - val_loss: 12.0522
Epoch 53/100
 - 1s - loss: 13.1676 - val_loss: 12.0490
Epoch 54/100
 - 1s - loss: 13.1640 - val_loss: 12.0585
Epoch 55/100
 - 1s - loss: 13.1561 - val_loss: 12.0456
Epoch 56/100
 - 1s - loss: 13.1540 - val_loss: 12.0435
Epoch 57/1

Epoch 86/100
 - 1s - loss: 13.2375 - val_loss: 12.2147
Epoch 87/100
 - 1s - loss: 13.2335 - val_loss: 12.2206
Epoch 88/100
 - 1s - loss: 13.2349 - val_loss: 12.2234
Epoch 89/100
 - 1s - loss: 13.2345 - val_loss: 12.2193
Epoch 90/100
 - 1s - loss: 13.2309 - val_loss: 12.2142
Epoch 91/100
 - 1s - loss: 13.2333 - val_loss: 12.2233
Epoch 92/100
 - 1s - loss: 13.2325 - val_loss: 12.2127
Epoch 93/100
 - 1s - loss: 13.2277 - val_loss: 12.2169
Epoch 94/100
 - 1s - loss: 13.2273 - val_loss: 12.2060
Epoch 95/100
 - 1s - loss: 13.2247 - val_loss: 12.2036
Epoch 96/100
 - 1s - loss: 13.2259 - val_loss: 12.2102
Epoch 97/100
 - 1s - loss: 13.2269 - val_loss: 12.1940
Epoch 98/100
 - 1s - loss: 13.2208 - val_loss: 12.1944
Epoch 99/100
 - 1s - loss: 13.2195 - val_loss: 12.2149
Epoch 100/100
 - 1s - loss: 13.2176 - val_loss: 12.2021
RMSE for  3_16_1  =  594.8251227813671
Starting training for :  3_16_2
Train on 35036 samples, validate on 8760 samples
Epoch 1/100
 - 1s - loss: 20.9706 - val_loss: 13.1474


...

 - 1s - loss: 13.2567 - val_loss: 12.1367
Epoch 55/100
 - 1s - loss: 13.2562 - val_loss: 12.1398
Epoch 56/100
 - 1s - loss: 13.2554 - val_loss: 12.1467
Epoch 57/100
 - 1s - loss: 13.2553 - val_loss: 12.1378
Epoch 58/100
 - 1s - loss: 13.2543 - val_loss: 12.1398
Epoch 59/100
 - 1s - loss: 13.2549 - val_loss: 12.1373
Epoch 60/100
 - 1s - loss: 13.2538 - val_loss: 12.1411
Epoch 61/100
 - 1s - loss: 13.2561 - val_loss: 12.1409
Epoch 62/100
 - 1s - loss: 13.2561 - val_loss: 12.1387
Epoch 63/100
 - 1s - loss: 13.2551 - val_loss: 12.1389
Epoch 64/100
 - 1s - loss: 13.2550 - val_loss: 12.1416
Epoch 65/100
 - 1s - loss: 13.2559 - val_loss: 12.1382
Epoch 66/100
 - 1s - loss: 13.2557 - val_loss: 12.1374
Epoch 67/100
 - 1s - loss: 13.2545 - val_loss: 12.1381
Epoch 68/100
 - 1s - loss: 13.2554 - val_loss: 12.1502
Epoch 69/100
 - 1s - loss: 13.2548 - val_loss: 12.1384
Epoch 70/100
 - 1s - loss: 13.2547 - val_loss: 12.1377
Epoch 71/100
 - 1s - loss: 13.2540 - val_loss: 12.1404
Epoch 72/100
 - 1s - lo

Train on 35033 samples, validate on 8759 samples
Epoch 1/100
 - 2s - loss: 35.5421 - val_loss: 14.3114
Epoch 2/100
 - 1s - loss: 14.3449 - val_loss: 12.6614
Epoch 3/100
 - 1s - loss: 13.6169 - val_loss: 12.3379
Epoch 4/100
 - 1s - loss: 13.4610 - val_loss: 12.2862
Epoch 5/100
 - 1s - loss: 13.3941 - val_loss: 12.3265
Epoch 6/100
 - 1s - loss: 13.3671 - val_loss: 12.2947
Epoch 7/100
 - 1s - loss: 13.3427 - val_loss: 12.3063
Epoch 8/100
 - 1s - loss: 13.3312 - val_loss: 12.3160
Epoch 9/100
 - 1s - loss: 13.3273 - val_loss: 12.3269
Epoch 10/100
 - 1s - loss: 13.3198 - val_loss: 12.3253
Epoch 11/100
 - 1s - loss: 13.3178 - val_loss: 12.3730
Epoch 12/100
 - 1s - loss: 13.3061 - val_loss: 12.3411
Epoch 13/100
 - 1s - loss: 13.3078 - val_loss: 12.3259
Epoch 14/100
 - 1s - loss: 13.3008 - val_loss: 12.3434
Epoch 15/100
 - 1s - loss: 13.3067 - val_loss: 12.4376
Epoch 16/100
 - 1s - loss: 13.3064 - val_loss: 12.4706
Epoch 17/100
 - 1s - loss: 13.3021 - val_loss: 12.3183
Epoch 18/100
 - 1s - loss

 - 1s - loss: 13.2718 - val_loss: 12.2005
Epoch 48/100
 - 1s - loss: 13.2722 - val_loss: 12.2024
Epoch 49/100
 - 1s - loss: 13.2691 - val_loss: 12.2008
Epoch 50/100
 - 1s - loss: 13.2678 - val_loss: 12.2160
Epoch 51/100
 - 1s - loss: 13.2668 - val_loss: 12.1996
Epoch 52/100
 - 1s - loss: 13.2658 - val_loss: 12.1996
Epoch 53/100
 - 1s - loss: 13.2670 - val_loss: 12.1840
Epoch 54/100
 - 1s - loss: 13.2635 - val_loss: 12.1965
Epoch 55/100
 - 1s - loss: 13.2639 - val_loss: 12.2266
Epoch 56/100
 - 1s - loss: 13.2631 - val_loss: 12.2058
Epoch 57/100
 - 1s - loss: 13.2672 - val_loss: 12.1947
Epoch 58/100
 - 1s - loss: 13.2661 - val_loss: 12.2022
Epoch 59/100
 - 1s - loss: 13.2634 - val_loss: 12.1871
Epoch 60/100
 - 1s - loss: 13.2659 - val_loss: 12.2050
Epoch 61/100
 - 1s - loss: 13.2671 - val_loss: 12.2010
Epoch 62/100
 - 1s - loss: 13.2661 - val_loss: 12.1884
Epoch 63/100
 - 1s - loss: 13.2673 - val_loss: 12.1862
Epoch 64/100
 - 1s - loss: 13.2659 - val_loss: 12.1903
Epoch 65/100
 - 1s - lo

 - 1s - loss: 13.2415 - val_loss: 12.2816
Epoch 95/100
 - 1s - loss: 13.2424 - val_loss: 12.2701
Epoch 96/100
 - 1s - loss: 13.2437 - val_loss: 12.2582
Epoch 97/100
 - 1s - loss: 13.2366 - val_loss: 12.2613
Epoch 98/100
 - 1s - loss: 13.2402 - val_loss: 12.2686
Epoch 99/100
 - 1s - loss: 13.2398 - val_loss: 12.2764
Epoch 100/100
 - 1s - loss: 13.2358 - val_loss: 12.2639
RMSE for  7_16_1  =  595.9492597562031
Starting training for :  7_16_2
Train on 35033 samples, validate on 8759 samples
Epoch 1/100
 - 2s - loss: 25.9242 - val_loss: 15.5182
Epoch 2/100
 - 1s - loss: 14.1294 - val_loss: 12.3518
Epoch 3/100
 - 1s - loss: 13.2912 - val_loss: 12.2233
Epoch 4/100
 - 1s - loss: 13.2660 - val_loss: 12.2002
Epoch 5/100
 - 1s - loss: 13.2554 - val_loss: 12.2013
Epoch 6/100
 - 1s - loss: 13.2453 - val_loss: 12.1452
Epoch 7/100
 - 1s - loss: 13.2396 - val_loss: 12.1456
Epoch 8/100
 - 1s - loss: 13.2435 - val_loss: 12.1143
Epoch 9/100
 - 1s - loss: 13.2625 - val_loss: 12.2676
Epoch 10/100
 - 1s - 

 - 1s - loss: 13.0964 - val_loss: 12.1040
Epoch 40/100
 - 1s - loss: 13.0991 - val_loss: 12.1193
Epoch 41/100
 - 1s - loss: 13.1111 - val_loss: 12.0614
Epoch 42/100
 - 1s - loss: 13.0954 - val_loss: 12.1495
Epoch 43/100
 - 1s - loss: 13.1152 - val_loss: 12.0446
Epoch 44/100
 - 1s - loss: 13.0846 - val_loss: 12.0257
Epoch 45/100
 - 1s - loss: 13.1098 - val_loss: 11.9835
Epoch 46/100
 - 1s - loss: 13.0821 - val_loss: 12.0881
Epoch 47/100
 - 1s - loss: 13.0966 - val_loss: 11.9710
Epoch 48/100
 - 1s - loss: 13.0723 - val_loss: 12.0668
Epoch 49/100
 - 1s - loss: 13.1123 - val_loss: 12.0022
Epoch 50/100
 - 1s - loss: 13.0774 - val_loss: 12.0341
Epoch 51/100
 - 1s - loss: 13.0949 - val_loss: 11.9940
Epoch 52/100
 - 1s - loss: 13.0734 - val_loss: 12.0164
Epoch 53/100
 - 1s - loss: 13.0679 - val_loss: 12.0358
Epoch 54/100
 - 1s - loss: 13.0966 - val_loss: 12.0020
Epoch 55/100
 - 1s - loss: 13.0761 - val_loss: 12.0023
Epoch 56/100
 - 1s - loss: 13.0689 - val_loss: 12.0041
Epoch 57/100
 - 1s - lo

In [83]:
loaded_scores = np.load('CNN_scores.npy')
sorted_scores = loaded_scores[loaded_scores[:, 0].argsort()]
sorted_scores


array([[5.67665677e+02, 3.00000000e+00, 1.60000000e+01, 2.00000000e+00],
       [5.68031571e+02, 5.00000000e+00, 1.60000000e+01, 2.00000000e+00],
       [5.68960990e+02, 7.00000000e+00, 1.60000000e+01, 2.00000000e+00],
       [5.69568609e+02, 5.00000000e+00, 4.00000000e+00, 3.00000000e+00],
       [5.69817485e+02, 5.00000000e+00, 1.60000000e+01, 3.00000000e+00],
       [5.71522916e+02, 3.00000000e+00, 4.00000000e+00, 3.00000000e+00],
       [5.72476524e+02, 7.00000000e+00, 1.60000000e+01, 3.00000000e+00],
       [5.72831235e+02, 3.00000000e+00, 1.60000000e+01, 3.00000000e+00],
       [5.87280308e+02, 5.00000000e+00, 2.00000000e+00, 2.00000000e+00],
       [5.90823542e+02, 7.00000000e+00, 4.00000000e+00, 3.00000000e+00],
       [5.90953181e+02, 7.00000000e+00, 2.00000000e+00, 2.00000000e+00],
       [5.92079519e+02, 7.00000000e+00, 4.00000000e+00, 2.00000000e+00],
       [5.94161760e+02, 3.00000000e+00, 2.00000000e+00, 2.00000000e+00],
       [5.94592548e+02, 5.00000000e+00, 4.00000000e